# P1) Path Tracking

This notebook tests your solution to problem 1 by using it to do joint space and task space inverse dynamics with acceleration and torque control.

You do not need to modify this code and it will not be marked. Edit `P1_tracking.py` instead.

In [ ]:
import sys
sys.path.append('.')

import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import plot_utils as plut
import time
import pinocchio as pin
from pinocchio.robot_wrapper import RobotWrapper
#import gepetto.corbaserver
import subprocess
import os
#import tsid

import talos_arm_conf as conf
from P1_tracking import compute_jsid_dv, compute_jsid_tau, compute_tsid_tau

Create a `RobotWrapper` specifying the URDF file describing the robot.

In [ ]:
robot = RobotWrapper.BuildFromURDF(conf.urdf, [os.path.join(conf.path, '../..')])
model = robot.model

Set initial configuration and velocity.

In [ ]:
q0 = conf.q0
v0 = np.zeros(robot.nv)

You can run a visualization of the robot if you install [meshcat](https://github.com/rdeits/meshcat-python). This is not necessary for the assignment, and has been commented out since it adds an additional package requirement.

In [ ]:
# skip visualization that requires MeshCat
#robot_display = pin.RobotWrapper.BuildFromURDF(conf.urdf, [os.path.join(conf.path, '../..')])
#Viewer = pin.visualize.MeshcatVisualizer
#viz = Viewer(robot_display.model, robot_display.collision_model, robot_display.visual_model)
#viz.initViewer(loadModel=True)
#viz.display(q0)

In [ ]:
#hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

Create empty arrays to store the simulation trajectories.

In [ ]:
N = conf.N_SIMULATION
q      = np.full((robot.nq, N + 1), np.nan)
v      = np.full((robot.nv, N + 1), np.nan)
dv     = np.full((robot.nv, N + 1), np.nan)
q_ref  = np.full((robot.nq, N), np.nan)
v_ref  = np.full((robot.nv, N), np.nan)
dv_ref = np.full((robot.nv, N), np.nan)
dv_des = np.full((robot.nv, N), np.nan)

Specify amplitude, phase and frequency of the sinusoidal joint trajectory to track.

In [ ]:
amp                  = np.array([0.2, 0.3, 0.4, 0.0, 0.0, 0.0, 0.0])               # amplitude
phi                  = np.array([0.0, 0.5 * np.pi, 0.0, 0.0, 0.0, 0.0, 0.0])       # phase
two_pi_f             = 2 * np.pi * np.array([1.0, 0.5, 0.3, 0.0, 0.0, 0.0, 0.0])   # frequency (time 2 PI)
two_pi_f_amp         = np.multiply(two_pi_f, amp)
two_pi_f_squared_amp = np.multiply(two_pi_f, two_pi_f_amp)

## P1 a) Joint space acceleration control
First we'll use your solution to P1 a) to do path tracking in joint space with acceleration control. At every step, your method `compute_dv` is called to compute joint acceleration. 

In [ ]:
import time
t = 0.0
dt = conf.dt
q[:, 0], v[:, 0] = q0, v0

for i in range(N):
    time_start = time.time()
    
    # set reference trajectory
    q_ref[:,i]  = q0 +  amp * np.sin(two_pi_f * t + phi)
    v_ref[:,i]  = two_pi_f_amp * np.cos(two_pi_f * t + phi)
    dv_ref[:,i] = -two_pi_f_squared_amp * np.sin(two_pi_f * t + phi)
    
    if i % conf.PRINT_N == 0:
        print("Time %.3f" % (t))
        
    # calculate dv
    Kp = conf.kp_posture * np.ones(robot.nv)
    Kd = 2.0 * np.sqrt(conf.kp_posture) * np.ones(robot.nv)
    dv[:,i]  = compute_jsid_dv(dv_ref[:,i], v[:,i], v_ref[:,i], q[:,i], q_ref[:,i], Kp, Kd)
    #dv_desired = dv_ref[:,i] - Kd * (v[:,i] - v_ref[:,i]) - Kp * (q[:,i] - q_ref[:,i])
    #dv[:,i] = dv_desired

    # numerical integration
    v_mean = v[:,i] + 0.5*dt*dv[:,i]
    v[:,i + 1] = v[:,i] + dt * dv[:,i]
    q[:,i + 1] = pin.integrate(model, q[:,i], dt * v_mean)
    t += conf.dt
    
    # skip vizualization that requires MeshCat
    #if i % conf.DISPLAY_N == 0:
        #viz.display(q[:,i])

    time_spent = time.time() - time_start
    if time_spent < conf.dt:
        time.sleep(conf.dt - time_spent)

# PLOT STUFF
plt_time = np.arange(0.0, N * conf.dt, conf.dt)


In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, q[i,:-1], label='q %i' % i)
    ax[i].plot(plt_time, q_ref[i,:], '--', label='q ref %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('q [rad]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, v[i,:-1], label='v %i ' % i)
    ax[i].plot(plt_time, v_ref[i,:], '--', label='v ref %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('v [rad/s]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, dv[i,:-1], label='dv '+str(i))
    ax[i].plot(plt_time, dv_ref[i,:], '--', label='dv ref %i' % i)
    ax[i].plot(plt_time, dv_des[i,:], ':', label='dv des %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('dv [rad/s^2]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

## P1 b) Joint space torque control
Next we'll use your solution to P1 b) to do path tracking in joint space with torque control. At every step, your method `compute_dv` is called to compute desired joint acceleration and your method `compute_tau` is called to calculate torque. 

In [ ]:
N = conf.N_SIMULATION
q      = np.full((robot.nq, N + 1), np.nan)
v      = np.full((robot.nv, N + 1), np.nan)
dv     = np.full((robot.nv, N + 1), np.nan)
tau    = np.full((robot.nq, N), np.nan)
q_ref  = np.full((robot.nq, N), np.nan)
v_ref  = np.full((robot.nv, N), np.nan)
dv_ref = np.full((robot.nv, N), np.nan)
dv_desired = np.full((robot.nv, N), np.nan)

In [ ]:
import time
t = 0.0
dt = conf.dt
q[:, 0], v[:, 0] = q0, v0

for i in range(N):
    time_start = time.time()
    
    # set reference trajectory
    q_ref[:,i]  = q0 +  amp * np.sin(two_pi_f * t + phi)
    v_ref[:,i]  = two_pi_f_amp * np.cos(two_pi_f * t + phi)
    dv_ref[:,i] = -two_pi_f_squared_amp * np.sin(two_pi_f * t + phi)
    
    if i % conf.PRINT_N == 0:
        print("Time %.3f" % (t))
        
    # calculate dv
    Kp = conf.kp_posture * np.ones(robot.nv)
    Kd = 2.0 * np.sqrt(conf.kp_posture) * np.ones(robot.nv)
    dv_desired[:,i]  = compute_jsid_dv(dv_ref[:,i], v[:,i], v_ref[:,i], q[:,i], q_ref[:,i], Kp, Kd)
    
    # calculate tau
    h = pin.rnea(model, robot.data, q[:,i], v[:,i], np.zeros_like(dv[:,i]))
    M = pin.crba(model, robot.data, q[:,i])
    tau[:,i] = compute_jsid_tau(dv_desired[:,i], M, h)
    
    # numerical integration
    dv[:,i] = pin.aba(model, robot.data, q[:,i], v[:,i], tau[:,i])
    v_mean = v[:,i] + 0.5*dt*dv[:,i]
    v[:,i + 1] = v[:,i] + dt * dv[:,i]
    q[:,i + 1] = pin.integrate(model, q[:,i], dt * v_mean)
    t += conf.dt
    
    # skip vizualization that requires MeshCat
    #if i % conf.DISPLAY_N == 0:
    #    viz.display(q[:,i])

    time_spent = time.time() - time_start
    if time_spent < conf.dt:
        time.sleep(conf.dt - time_spent)

# PLOT STUFF
plt_time = np.arange(0.0, N * conf.dt, conf.dt)


In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, q[i,:-1], label='q %i' % i)
    ax[i].plot(plt_time, q_ref[i,:], '--', label='q ref %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('q [rad]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, v[i,:-1], label='v %i ' % i)
    ax[i].plot(plt_time, v_ref[i,:], '--', label='v ref %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('v [rad/s]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, dv[i,:-1], label='dv '+str(i))
    ax[i].plot(plt_time, dv_ref[i,:], '--', label='dv ref %i' % i)
    ax[i].plot(plt_time, dv_des[i,:], ':', label='dv des %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('dv [rad/s^2]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(robot.nv, 1, figsize=(10, 20))
for i in range(robot.nv):
    ax[i].plot(plt_time, tau[i,:], label='tau %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('tau [Nm]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()

## P1 c) Task space torque control
Next we'll use your solution to P1 c) to do path tracking in task space with torque control.

In [ ]:
N = conf.N_SIMULATION
q      = np.full((robot.nq, N + 1), np.nan)
v      = np.full((robot.nv, N + 1), np.nan)
dv     = np.full((robot.nv, N + 1), np.nan)
x      = np.full((3, N + 1), np.nan)
tau    = np.full((robot.nq, N), np.nan)
x_ref  = np.full((3, N), np.nan)


In [ ]:
amp                  = np.array([0.05, 0.05, 0.05])               # amplitude
phi                  = np.array([0.0, 0.0, 0.0])       # phase
two_pi_f             = 2 * np.pi * np.array([0.01, 0.01, 0.01])   # frequency (time 2 PI)
two_pi_f_amp         = np.multiply(two_pi_f, amp)
two_pi_f_squared_amp = np.multiply(two_pi_f, two_pi_f_amp)

In [ ]:
from numpy.linalg import norm, solve
data = robot.data

JOINT_ID = 6

pin.forwardKinematics(model,data,q[:,-1])
x0     = np.array([0.15,0.15,0.15])
eps    = 1e-4
DT     = 1e-1
damp   = 1e-6

q0 = np.array([2.298, 3.581, 2.318, 5.749, 0.797, 1.253, 0.   ])
q[:, 0], v[:, 0] = q0, v0
x[:, 0] = x0

i=0
t = 0.
for i in range(N):
    x_ref[:,i]  = x0 + amp * np.sin(two_pi_f * t + phi)
    oMdes = pin.SE3(np.eye(3), x_ref[:,i])

    pin.forwardKinematics(model,data,q[:,i])
    dMi = oMdes.actInv(data.oMi[JOINT_ID])
    err = pin.log(dMi).vector

    J = pin.computeJointJacobian(model,data,q[:,i],JOINT_ID)
    pin.computeJointJacobians(model,data,q[:,i])
    pin.computeJointJacobiansTimeVariation(model,data,q[:,i],v[:,i])
    dJ = pin.getJointJacobianTimeVariation(model,data,JOINT_ID,pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

    J_pinv = np.linalg.pinv(J)
    Kp = conf.kp_posture * np.ones(6)
    ddx_desired = -Kp * err

    h = pin.rnea(model, data, q[:,i], v[:,i], np.zeros_like(v[:,i]))
    M = pin.crba(model, data, q[:,i])
    tau[:,i] = compute_tsid_tau(ddx_desired, J, dJ, v[:,i], M, h)
    
    dv[:,i] = pin.aba(model, data, q[:,i], v[:,i], tau[:,i])

    v_mean = v[:,i] + 0.5*DT*dv[:,i]
    v[:,i + 1] = v[:,i] + dt * dv[:,i]
    q[:,i+1] = pin.integrate(model, q[:,i], DT * v_mean)
    x[:,i+1] = data.oMi[JOINT_ID].translation
    
    if i % conf.PRINT_N == 0:
        print("Time %.3f" % (t))

    # skip vizualization which requires MeshCat
    #if not i % 10:
    #    viz.display(q[:,i])
    i += 1
    t += DT

# PLOT STUFF
plt_time = np.arange(0.0, N * conf.dt, conf.dt)


In [ ]:
(f, ax) = plut.create_empty_figure(3, 1, figsize=(10, 20))
for i in range(3):
    ax[i].plot(plt_time, x[i,:-1], label='q %i' % i)
    ax[i].plot(plt_time, x_ref[i,:], '--', label='q ref %i' % i)
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('q [rad]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
plt.show()